In [1]:
import os 

import numba
import math

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import erlang

In [2]:
def get_linear_burst(t,tlim,a):
    if t<tlim:
        return 1.0
    elif t>=tlim:
        return a*t

In [6]:
def taul_to_minutes(t,n,k):
    return t*(float(k)/float(n))*23.0
    

In [7]:
def minutes_to_taul(t,n,k):
    return t*(float(n)/float(k))/23.0

In [3]:
@numba.jit
def monod_growth_law(n,kn,gnmax=0.034):
    return gnmax*n/(kn+n)

In [ ]:
@numba.jit    
def system_evolution_nlg(delta_t,Nb0,B,P,N,k,nd,nu,n0,doubling_time,niter):
    
    # Volume of the system
    V = float(Nb0)/float(B)
    
    # How many phages do we have in this voalume according to the initial concentration?
    Np0 = int(P*V)
    
    # how many units of nutrients do we have?
    nutrients = int(n0*V)
    gnmax = np.log(2)/doubling_time
    
    # Initialization of vectors that contain information about bacteria
    states = np.zeros(Nb0,dtype=np.int8)
    infection_time = np.zeros(Nb0, dtype=np.float16)
    sizes = np.zeros(Nb0)
    Nt = np.zeros(Nb0, dtype=np.int8)
    
    # Initialization of vectors that grow with time steps
    time = []
    phages = []
    bacteria = []
    #nut = []
    
    # We store the first values of the concentratioin of bacteria and phage
    bacteria.append(float(Nb0)/float(V))
    phages.append(float(Np0)/float(V))
    
    # We initialize time
    t = 0.0
    time.append(t)
    
    # We initialize the number of phages
    Np = Np0
    
    # probability of new timer molecules for infected bacteria
    prob_new_timer = k*delta_t
    
    iteration = 0
    
    for n in range(0,niter):
        
        # we count the infected and uninfected bacteria
        n_infected = len(np.where(states==1)[0])
        n_uninfected = len(np.where(states==0)[0])
        Nb = n_infected + n_uninfected
        
        # Probability of infection at this time step
        prob_infection = (nu*delta_t*(float(Np)/V))
        prob_superinfection = (nu*delta_t*(float(Np)/V))
        sum_probs = prob_superinfection + prob_new_timer
        
        # We iterate over the already infected cells
        for i in np.where(states==1)[0]:
            r = np.random.uniform()
            if r < prob_new_timer:
                Nt[i] = Nt[i] + 1
                #print('New timer')
                if Nt[i] >= N:
                    tau = t - infection_time[i]
                    Np = Np + int(get_linear_burst(tau,15.0,1.0))
                    states[i] = 2
                    
                    # We delete the dead cells from all the variables dependent on the number of bacteria
                    np.delete(states,i)
                    np.delete(infection_time,i)
                    np.delete(sizes,i)
                    np.delete(Nt,i)
                   
            elif prob_new_timer < r < sum_probs:
                if Np > 0:
                    Np = Np - 1
                else:
                    Np = 0
                    
                if Nt[i] < nd:
                    Nt[i] = 0 
                elif Nt[i] >= nd:
                    Nt[i] = Nt[i] - nd
            
        
        # We iterate over the non infected cells to see if they get infected
        non_infected_indices = np.where(states==0)[0]
        # the growth rate is inversely proportional to gn
        n = float(nutrients)/V
        alpha = 1.0/monod_growth_law(n=n,kn=n0/5.0,gnmax=gnmax)
        sumprobs = prob_infection + alpha*delta_t
        for i in non_infected_indices:
            r = np.random.uniform()
            if r<prob_infection:
                #print('first infection')
                # They are infected now
                states[i] = 1
                Np = Np-1
                # Wr store their infection time
                infection_time[i] = t
            elif prob_infection<r<sumprobs:
                nutrients = nutrients - 1   
                # We re-initialize all vectors that depend on Nb
                states = np.append(states,0)
                infection_time = np.append(infection_time,0.0)
                Nt = np.append(Nt,0)
                    
        Nb = len(np.where(states==0)[0]) + len(np.where(states==1)[0])
        
        t = t + delta_t
        phages.append(float(Np)/V)
        time.append(t)
        bacteria.append(float(len(np.where(states==0)[0]) + len(np.where(states==1)[0]))/V)
        #nut.append(n)
        
        iteration = iteration + 1
        if np.mod(iteration,1000)==0.0:
            print(Nb)
    
    
    return time,phages,bacteria,nut

In [ ]:
# The time units are (n/k) which is tau_l
# ULRIK'S PARAMETERS

delta_t = 0.01
Nb0 = 50
B = 1.0e6
P0 = 0
N = 60
k = 6.0
nd = 30
nu = 5.0e-9
n0 = 1.0e6

# The doubling time in minutes
doubling_time_mins = 20.0
doubling_time = minutes_to_taul(doubling_time_mins,N,k)

niter = 1000